In [1]:
import mxnet as mx
from mxnet import init, gluon, nd, autograd, image
from mxnet.gluon import nn
from mxnet.gluon.data import vision
from mxnet.gluon.model_zoo import vision as models
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import h5py
import os
from glob import glob
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

ctx = mx.gpu()

In [2]:
def build_model():
    net = nn.Sequential()
    with net.name_scope():
        net.add(nn.BatchNorm())
        net.add(nn.Dense(1024))
        net.add(nn.BatchNorm())
        net.add(nn.Activation('relu'))
        net.add(nn.Dropout(0.5))
        net.add(nn.Dense(512))
        net.add(nn.BatchNorm())
        net.add(nn.Activation('relu'))
        net.add(nn.Dropout(0.5))
        net.add(nn.Dense(120))
    net.initialize(ctx=ctx)
    return net

def accuracy(output, labels):
    return nd.mean(nd.argmax(output, axis=1) == labels).asscalar()

def evaluate(net, data_iter):
    loss, acc, n = 0., 0., 0.
    steps = len(data_iter)
    for data, label in data_iter:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        output = net(data)
        acc += accuracy(output, label)
        loss += nd.mean(softmax_cross_entropy(output, label)).asscalar()
    return loss/steps, acc/steps

In [3]:
synset = list(pd.read_csv('sample_submission.csv').columns[1:])
n = len(glob('Images/*/*.jpg'))

y = nd.zeros((n,))
for i, file_name in tqdm(enumerate(glob('Images/*/*.jpg')), total=n):
    y[i] = synset.index(file_name.split('/')[1][10:].lower())
    nd.waitall()

100%|██████████| 20580/20580 [00:01<00:00, 18391.75it/s]


In [4]:
features = [nd.load('features_incep.nd')[0], nd.load('features_res.nd')[0]]
features = nd.concat(*features, dim=1)

In [5]:
data_iter_train = gluon.data.DataLoader(gluon.data.ArrayDataset(features, y), 128, shuffle=True)

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
net = build_model()
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': 1e-3})

In [6]:
epochs = 100
batch_size = 128

for epoch in range(epochs):
    if epoch == 50:
        trainer.set_learning_rate(1e-4)
    if epoch == 80:
        trainer.set_learning_rate(1e-5)
    train_loss = 0.
    train_acc = 0.
    steps = len(data_iter_train)
    for data, label in data_iter_train:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)
        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output, label)
    if epoch%10 == 0:
        print("Epoch %d. loss: %.4f, acc: %.2f%%" % (epoch, train_loss/steps, train_acc/steps*100))
print("Epoch %d. loss: %.4f, acc: %.2f%%" % (epoch, train_loss/steps, train_acc/steps*100))
evaluate(net, data_iter_train)

Epoch 0. loss: 1.1068, acc: 76.33%
Epoch 10. loss: 0.1229, acc: 95.91%
Epoch 20. loss: 0.0781, acc: 97.35%
Epoch 30. loss: 0.0525, acc: 98.30%
Epoch 40. loss: 0.0452, acc: 98.50%
Epoch 50. loss: 0.0286, acc: 99.02%
Epoch 60. loss: 0.0177, acc: 99.45%
Epoch 70. loss: 0.0127, acc: 99.53%
Epoch 80. loss: 0.0119, acc: 99.64%
Epoch 90. loss: 0.0101, acc: 99.66%
Epoch 99. loss: 0.0094, acc: 99.67%


(0.002932575075505811, 0.99820458074534157)

In [7]:
model_names = ['inceptionv3', 'resnet152_v1']
features_test = [nd.load('features_test_%s.nd' % model_name)[0] for model_name in model_names]
features_test = nd.concat(*features_test, dim=1)

output = nd.softmax(net(features_test.as_in_context(ctx))).asnumpy()

In [8]:
df_pred = pd.read_csv('sample_submission.csv')

for i, c in enumerate(df_pred.columns[1:]):
    df_pred[c] = output[:,i]

df_pred.to_csv('pred.csv', index=None)